In [1]:
import torch
import torch_geometric
import pandas as pd

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
from tqdm.notebook import tqdm
import numpy as np

In [2]:
import os
for dirname, _, filenames in os.walk('/higgs/icecube'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/higgs/icecube/test_meta.parquet
/higgs/icecube/train_meta.parquet
/higgs/icecube/sensor_geometry.csv
/higgs/icecube/sample_submission.parquet
/higgs/icecube/test/batch_661.parquet
/higgs/icecube/train/batch_420.parquet
/higgs/icecube/train/batch_158.parquet
/higgs/icecube/train/batch_312.parquet
/higgs/icecube/train/batch_43.parquet
/higgs/icecube/train/batch_235.parquet
/higgs/icecube/train/batch_507.parquet
/higgs/icecube/train/batch_231.parquet
/higgs/icecube/train/batch_47.parquet
/higgs/icecube/train/batch_503.parquet
/higgs/icecube/train/batch_424.parquet
/higgs/icecube/train/batch_316.parquet
/higgs/icecube/train/batch_239.parquet
/higgs/icecube/train/batch_154.parquet
/higgs/icecube/train/batch_150.parquet
/higgs/icecube/train/batch_428.parquet
/higgs/icecube/train/batch_94.parquet
/higgs/icecube/train/batch_407.parquet
/higgs/icecube/train/batch_335.parquet
/higgs/icecube/train/batch_64.parquet
/higgs/icecube/train/batch_212.parquet
/higgs/icecube/train/batch_520.parquet
/hig

In [3]:
train_data=pd.read_parquet('/higgs/icecube/train/batch_1.parquet')

train_data_event=train_data.loc[24]
#for i in train_data_event['charge']:
    #print(i)
train_data.loc

In [4]:
# train_data.index

In [5]:
# def charge_rank(event):
    
#     for i in event['charge']

In [6]:
# prev=[]
# events=0
# for i in train_data.index:
    
#     prev.append(i)
    
# for i in prev:
    
#     if prev[i+1] != prev[i]:
#         events +=1
        
    
        
    
    
# events  

In [52]:
train_data

,sensor_id,time,charge,auxiliary
event_id,,,,
24,3918,5928,1.325,True
24,4157,6115,1.175,True
24,3520,6492,0.925,True
24,5041,6665,0.225,True
24,2948,8054,1.575,True
...,...,...,...,...
3266196,1961,17617,0.975,True
3266196,36,17705,0.625,True
3266196,356,18192,0.725,True


In [9]:
!pip install wandb

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 6.8 MB 8.7 MB/s eta 0:00:01
     |████████████████████████████████| 289 kB 47.1 MB/s eta 0:00:01
     |████████████████████████████████| 143 kB 42.6 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import wandb
import gc

In [11]:
torch.manual_seed(108)

In [12]:
#Hyperparameter Block


In [13]:
df_sensor_geometry = pd.read_csv(f"/higgs/icecube/sensor_geometry.csv")

In [14]:
print(df_sensor_geometry)

      sensor_id       x       y       z
0             0 -256.14 -521.08  496.03
1             1 -256.14 -521.08  479.01
2             2 -256.14 -521.08  461.99
3             3 -256.14 -521.08  444.97
4             4 -256.14 -521.08  427.95
...         ...     ...     ...     ...
5155       5155  -10.97    6.72 -472.39
5156       5156  -10.97    6.72 -479.39
5157       5157  -10.97    6.72 -486.40
5158       5158  -10.97    6.72 -493.41
5159       5159  -10.97    6.72 -500.73

[5160 rows x 4 columns]


In [15]:
#Pull single event from a batch
batch_id = 10
batch = pd.read_parquet(f"/higgs/icecube/train/batch_{batch_id}.parquet")

In [16]:
event_id = 29296372

In [17]:
times = np.sort(batch.loc[event_id].time.to_numpy())
print(times[0],times[len(times) - 1])
print(times)

29807 40084
[29807 30746 31649 31890 32030 32609 32612 32647 33545 33562 33683 33721
 33814 33939 33962 33962 33967 33981 33986 33993 34031 34132 34208 34234
 34838 34853 34981 35029 35142 35861 35957 36081 36441 36449 36493 38172
 38294 38357 38874 39008 39364 39768 39873 40084]


In [18]:
print("First Sensor: \n" ,batch.loc[event_id].iloc[0],"\n Last Sensor: \n" ,batch.loc[event_id].iloc[len(batch.loc[event_id]) - 1])


First Sensor: 
 sensor_id     2599
time         29807
charge       1.125
auxiliary     True
Name: 29296372, dtype: object 
 Last Sensor: 
 sensor_id     3102
time         40084
charge       1.475
auxiliary     True
Name: 29296372, dtype: object


In [19]:
full_event = pd.merge(batch.loc[event_id], df_sensor_geometry, on="sensor_id")
print(full_event)

    sensor_id   time  charge  auxiliary       x       y       z
0        2599  29807   1.125       True -156.23   43.37  177.44
1         711  30746   1.125       True  282.18 -325.74 -369.61
2          14  31649   1.225       True -256.14 -521.08  257.74
3        5070  31890   0.725       True   -9.68  -79.50 -296.58
4        5076  32030   0.625       True   -9.68  -79.50 -338.63
5         767  32609   1.525       True  405.83 -306.38 -304.84
6         527  32612   1.325       True  -88.05 -384.30 -299.96
7         767  32647   0.525       True  405.83 -306.38 -304.84
8        3741  33545   1.025       True  -66.70  276.92  140.39
9         159  33562   0.525       True   -9.13 -481.74 -164.57
10        280  33683   0.975      False  237.78 -442.42 -179.37
11       5094  33721   0.275       True   -9.68  -79.50 -464.79
12        279  33814   1.175      False  237.78 -442.42 -162.34
13        282  33939   1.175      False  237.78 -442.42 -213.41
14        338  33962   0.575      False 

In [20]:
# print((full_event.charge * full_event.x).mean)
x_CM = np.mean((full_event.charge * full_event.x).to_numpy())
y_CM = np.mean((full_event.charge * full_event.y).to_numpy())
z_CM = np.mean((full_event.charge * full_event.z).to_numpy())
print(x_CM,y_CM,z_CM)

97.76757965373722 -97.07294967507096 -91.56235282662918


In [21]:
# Maybe we want to just care about the weighted auxilaries?
# TODO, later

In [22]:
# Params to pass: first sensor, last sensor, [x,y,z], all sensors
# Bonus Params: Center of Charge(weighted average of charge location)


In [23]:
#params: az_true, zen_true, az_pred, zen_pred
#returns: dist := mean over the angular disance(s) in radians
#Defining Angular Loss Function

def angular_dist(az_true, zen_true, az_pred, zen_pred):
    if not (torch.all(torch.isfinite(az_true))  and
            torch.all(torch.isfinite(zen_true)) and
            torch.all(torch.isfinite(az_pred)) and
            torch.all(torch.isfinite(zen_pred))
           ):
        raise ValueError("All arguments must be finite")
    
    # pre-compute all sine and cosine values
    sa1 = torch.sin(az_true)
    ca1 = torch.cos(az_true)
    sz1 = torch.sin(zen_true)
    cz1 = torch.cos(zen_true)
    
    
    
    sa2 = torch.sin(az_pred)
    ca2 = torch.cos(az_pred)
    sz2 = torch.sin(zen_pred)
    cz2 = torch.cos(zen_pred)
    
    # scalar product of the two cartesian vectors (x = sz*ca, y = sz*sa, z = cz)
    scalar_prod = sz1*sz2*(ca1*ca2 + sa1*sa2) + (cz1*cz2)
    
    # scalar product of two unit vectors is always between -1 and 1, this is against nummerical instability
    # that might otherwise occure from the finite precision of the sine and cosine functions
    scalar_prod =  torch.clamp(scalar_prod, -1, 1)
    
    # convert back to an angle (in radian)
    return torch.abs(torch.acos(scalar_prod))
    

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as checkpoint
import math
from timm.models.layers import drop_path, trunc_normal_
import math
import torch.utils.checkpoint as checkpoint
from typing import Any, Callable, List, Optional, Sequence, Tuple, Union
from torch import Tensor, LongTensor
from timm.models.layers import drop_path, trunc_normal_

In [25]:
train_meta = pd.read_parquet("/higgs/icecube/train_meta.parquet")

In [26]:
print(train_meta.loc[train_meta.event_id == event_id])

         batch_id  event_id  first_pulse_index  last_pulse_index   azimuth  \
1800000        10  29296372                  0                43  3.517532   

           zenith  
1800000  1.616892  


In [51]:
train_meta.loc[train_meta.batch_id==10]

,batch_id,event_id,first_pulse_index,last_pulse_index,azimuth,zenith
1800000,10,29296372,0,43,3.517532,1.616892
1800001,10,29296374,44,132,5.775634,0.199164
1800002,10,29296414,133,180,3.715111,0.769641
1800003,10,29296416,181,241,2.613874,1.399564
1800004,10,29296437,242,310,4.110431,1.054067
...,...,...,...,...,...,...
1999995,10,32567581,33242816,33242866,0.353034,1.329428
1999996,10,32567639,33242867,33242921,3.437011,1.079144
1999997,10,32567659,33242922,33242992,1.112257,2.569093
1999998,10,32567680,33242993,33243195,0.898184,2.139322


In [45]:
full_event1 = pd.merge(batch, df_sensor_geometry, on="sensor_id")
print(full_event1)

          sensor_id   time  charge  auxiliary       x       y       z
0              2599  29807   1.125       True -156.23   43.37  177.44
1               711  30746   1.125       True  282.18 -325.74 -369.61
2                14  31649   1.225       True -256.14 -521.08  257.74
3              5070  31890   0.725       True   -9.68  -79.50 -296.58
4              5076  32030   0.625       True   -9.68  -79.50 -338.63
...             ...    ...     ...        ...     ...     ...     ...
33243253        570  16812   0.675       True   35.54 -364.83  -12.91
33243254        631  17481   1.125       True  158.94 -345.23  -27.81
33243255       4134  17767   0.875       True -268.90  354.24 -417.59
33243256       4973  18139   0.775       True  113.19  -60.47 -462.47
33243257       4973  18152   1.025       True  113.19  -60.47 -462.47

[33243258 rows x 7 columns]


In [27]:
batch

,sensor_id,time,charge,auxiliary
event_id,,,,
29296372,2599,29807,1.125,True
29296372,711,30746,1.125,True
29296372,14,31649,1.225,True
29296372,5070,31890,0.725,True
29296372,5076,32030,0.625,True
...,...,...,...,...
32567683,570,16812,0.675,True
32567683,631,17481,1.125,True
32567683,4134,17767,0.875,True


In [28]:
class LSTMNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMNet, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [29]:
input_size = 5160*6
model = LSTMNet(input_size, 40, 3, 2)
print(model)

LSTMNet(
  (lstm): LSTM(30960, 40, num_layers=3, batch_first=True)
  (fc): Linear(in_features=40, out_features=2, bias=True)
)


In [30]:
import tqdm

In [31]:
def getData(n):
    return 1, [0,0]

In [37]:
epochs = 100
eta = 0.005
optimizer = torch.optim.Adam(model.parameters(), lr=eta)
for i in tqdm.tqdm(range(epochs)):
    X, y = data_loader.get_xy(split="train")X['xyz'][0][:3], X['time'][0][:3], X['charge'][0][:3], X['auxiliary'][0][:3]
    optimizfer.zero_grad(set_to_none=True)
    optimizer.step()

SyntaxError: invalid syntax (1102352811.py, line 5)

In [33]:
criterion = nn.MSELoss()

In [34]:
#figure out data input into model for training

In [43]:
# import torch
# import tqdm
# import torch
# from torch.utils.data import Dataset, DataLoader

# epochs = 100
# eta = 0.005
# optimizer = torch.optim.Adam(model.parameters(), lr=eta)
# data_loader = DataLoader(train_meta, batch_size=32, shuffle=True)
# # Define your loss function (example: CrossEntropyLoss for classification)
# criterion = torch.nn.CrossEntropyLoss()

# for i in tqdm.tqdm(range(epochs)):
#     # Load the training data
#     X, y = data_loader.get_xy(split="train")
    
#     # Example of how you might access specific parts of the data
#     xyz_data = X['xyz'][0][:3]
#     time_data = X['time'][0][:3]
#     charge_data = X['charge'][0][:3]
#     auxiliary_data = X['auxiliary'][0][:3]
    
#     optimizer.zero_grad(set_to_none=True)
    
#     # Forward pass
#     outputs = model(xyz_data, time_data, charge_data, auxiliary_data)
    
#     # Compute the loss
#     loss = criterion(outputs, y)
    
#     # Backward pass
#     loss.backward()
    
#     # Optimization step
#     optimizer.step()


In [35]:
seq_length = 10
num_samples = 1000

X = full_event

# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32).unsqueeze(-1)
y = torch.tensor(y, dtype=torch.float32).unsqueeze(-1)

ValueError: could not determine the shape of object type 'DataFrame'

In [36]:
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
#     outputs = model(X)
    optimizer.zero_grad()

    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 20 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print('Finished Training')

NameError: name 'optimizer' is not defined

In [35]:
print(train_meta.loc[train_meta.event_id == event_id])

         batch_id  event_id  first_pulse_index  last_pulse_index   azimuth  \
1800000        10  29296372                  0                43  3.517532   

           zenith  
1800000  1.616892  


In [36]:
events = train_meta.event_id.to_numpy()

In [61]:
#get dataframe of event 
#params: dataset is dataset of metadata

def getEvent(dataset, event_id, batch):
    event_meta = dataset.loc[dataset.event_id == event_id]
    fpi = int(event_meta.first_pulse_index.values[0])
    lpi = int(event_meta.last_pulse_index.values[0])
    azi_true = float(event_meta.azimuth.values[0])
    zen_true = float(event_meta.zenith.values[0])
    batch_number = int(event_meta.batch_id.values[0])
#     print(event_meta)
#     print("Fpi: ", fpi)
#     print("\nLpi: ", lpi)
    event_data = batch.iloc[fpi:lpi,0:6]
    
    return pd.merge(event_data, df_sensor_geometry, on="sensor_id"),[azi_true, zen_true]


In [69]:
# batch.iloc[0:43,0:6]

In [70]:
# for i in range(len(events)):
event_id = 29296372
X, y = getEvent(train_meta, event_id, batch)

In [71]:
print(X,y)

    sensor_id   time  charge  auxiliary       x       y       z
0        2599  29807   1.125       True -156.23   43.37  177.44
1         711  30746   1.125       True  282.18 -325.74 -369.61
2          14  31649   1.225       True -256.14 -521.08  257.74
3        5070  31890   0.725       True   -9.68  -79.50 -296.58
4        5076  32030   0.625       True   -9.68  -79.50 -338.63
5         767  32609   1.525       True  405.83 -306.38 -304.84
6         527  32612   1.325       True  -88.05 -384.30 -299.96
7         767  32647   0.525       True  405.83 -306.38 -304.84
8        3741  33545   1.025       True  -66.70  276.92  140.39
9         159  33562   0.525       True   -9.13 -481.74 -164.57
10        280  33683   0.975      False  237.78 -442.42 -179.37
11       5094  33721   0.275       True   -9.68  -79.50 -464.79
12        279  33814   1.175      False  237.78 -442.42 -162.34
13        282  33939   1.175      False  237.78 -442.42 -213.41
14        338  33962   0.575      False 

In [72]:
print(events)

[        24         41         59 ... 2147483617 2147483626 2147483627]


In [73]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

2024-06-09 20:47:22.671518: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [63]:
batch_id = 1
batch_1 = pd.read_parquet(f"/higgs/icecube/train/batch_{batch_id}.parquet")

In [55]:
# Generate Data. first try, batch_id = 1
batch_1_events = train_meta[train_meta.batch_id == 1]

In [2]:
b1_events_arr = batch_1_events.event_id.to_numpy()

NameError: name 'batch_1_events' is not defined

In [66]:
print(b1_events_arr)
len(b1_events_arr)


[     24      41      59 ... 3266192 3266195 3266196]


200000

In [64]:
X = []
y = []
for i in tqdm.tqdm(range(10_000)):
    x, u = getEvent(train_meta, b1_events_arr[i], batch_1)
#     print(x,u)
    X.append(x)
    y.append(u)

  5%|▍         | 477/10000 [05:28<1:49:27,  1.45it/s]


KeyboardInterrupt: 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Parameters
seq_length = 6  # Each sequence is of length 6
input_features = 5160  # Each input vector has 5160 features
num_samples = 1000  # Total of 1000 sequences
output_size = 2  # Output consists of two continuous float values


# Split the data into training and test sets
train_size = int(0.8 * num_samples)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Define the LSTM model
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(seq_length, input_features)))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(output_size))  # No activation function for regression output

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, batch_size=32, verbose=1)

# Plot training history
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()

# # Evaluate the model
# test_loss, test_mae = model.evaluate(X_test, y_test, verbose=1)
# print(f'Test Loss: {test_loss}')
# print(f'Test MAE: {test_mae}')

# # Make predictions
# predictions = model.predict(X_test)

# # Example of predicted vs actual for the first test sample
# print(f'Predicted: {predictions[0]}, Actual: {y_test[0]}')


In [ ]:
# model = Sequential()
# model.add(LSTM(5160*6, activation='relu', input_shape=(seq_length, 1)))
# model.add(Linear(40, activation='relu', input_shape=(seq_length, 1))
# model.add(Dense(1))
# model.compile(optimizer='adam', loss='mse')

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load metadata
train_meta1 = pd.read_parquet('/higgs/icecube/train_meta.parquet')
train_meta = train_meta1.loc[train_meta1.batch_id==1]
print("Succesfully downloaded metadata")
# Load a subset of event data
batch = pd.read_parquet('/higgs/icecube/train/batch_1.parquet')

print("Succesfully downloaded batch 1")
# Select a subset of the data for simplicity
subset_events = train_meta.sample(frac=0.1, random_state=42)
subset_event_ids = subset_events['event_id'].unique()
subset_data = train_meta[train_meta.batch_id == 1].isin(subset_event_ids)

# Feature engineering
def create_features(event_data):
    features = event_data.groupby('event_id').agg({
        'time': ['min', 'max', 'mean', 'std'],
        'charge': ['sum', 'mean', 'std'],
        'sensor_id': 'count'
    }).reset_index()
    features.columns = ['event_id', 'time_min', 'time_max', 'time_mean', 'time_std',
                        'charge_sum', 'charge_mean', 'charge_std', 'pulse_count']
    return features

# Create training features
train_features = create_features(subset_data)
train_data = subset_events.merge(train_features, on='event_id')

# Train/Validation split
X = train_data.drop(['azimuth', 'zenith'], axis=1)
y_azimuth = train_data['azimuth']
y_zenith = train_data['zenith']

X_train, X_val, y_azimuth_train, y_azimuth_val, y_zenith_train, y_zenith_val = train_test_split(
    X, y_azimuth, y_zenith, test_size=0.2, random_state=42
)

# Train Linear Regression model for azimuth
azimuth_model_lr = LinearRegression()
azimuth_model_lr.fit(X_train, y_azimuth_train)

# Train Decision Tree model for zenith
zenith_model_dt = DecisionTreeRegressor()
zenith_model_dt.fit(X_train, y_zenith_train)

# Validation
y_azimuth_pred = azimuth_model_lr.predict(X_val)
y_zenith_pred = zenith_model_dt.predict(X_val)

# Evaluation
azimuth_mse = mean_squared_error(y_azimuth_val, y_azimuth_pred)
zenith_mse = mean_squared_error(y_zenith_val, y_zenith_pred)

print(f'MSE for Azimuth Prediction: {azimuth_mse}')
print(f'MSE for Zenith Prediction: {zenith_mse}')

# Prediction on test data (example)
# test_batch = pd.read_parquet('test_batch_1.parquet')
# test_features = create_features(test_batch)
# test_azimuth_pred = azimuth_model_lr.predict(test_features)
# test_zenith_pred = zenith_model_dt.predict(test_features)


Succesfully downloaded metadata
Succesfully downloaded batch 1


KeyError: "Column(s) ['charge', 'sensor_id', 'time'] do not exist"

In [3]:
subset_events = train_meta.sample(frac=0.1, random_state=42)
subset_events

,batch_id,event_id,first_pulse_index,last_pulse_index,azimuth,zenith
94755642,474,1542233455,25547388,25547437,3.099153,2.798651
125331818,627,2039855459,21242434,21242460,5.270917,2.229669
64300562,322,1046563400,16393048,16393124,5.271733,0.844585
120537183,603,1961800589,22630132,22630199,5.494447,2.387042
21030563,106,342181158,5084599,5084736,5.550612,1.643652
...,...,...,...,...,...,...
23764305,119,386678316,26762410,26762477,1.373408,0.684191
57948905,290,943193611,24104779,24104839,3.401873,1.920408
71709396,359,1167104626,17411154,17411210,5.838817,1.561619
33482878,168,544897581,13780454,13781315,4.033885,1.055398
